In [1]:
import pandas as pd
from pymongo import MongoClient
import random

from mongo_aggregation_verbs import *

mongo_client = MongoClient('18.236.138.158', 27016)
database_reference = mongo_client.twitter

In [2]:
database_reference.collection_names()

collection_reference = database_reference.instructor_test_group

test_group = database_reference.instructor_test_group

In [3]:
match_empty_url_arrays = { MATCH : { "entities.urls" : [] } }

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text" }
    ]
))

[{'text': 11121}]

In [4]:
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags } } }
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

([{'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')}],
 1835)

In [5]:
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

[ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6df39136dd5f0001691533'),
 ObjectId('5a6df39136dd5f0001691533')]

In [6]:
not_in_bad_ids = { "$nin" : bad_ids }

In [7]:
not_in_bad_ids_and_no_url = { 
    "_id"           : not_in_bad_ids, 
    "entities.urls" : []
}

just_the_text = {
    "text" : 1,
    "_id"  : 0
}

In [8]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

{'text': "@SincerelyLegit Lol why at night? If it's boring I'll fall asleep"}

In [9]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [10]:
len(tweet_text)

11102

In [11]:
tweet_text.head()

,text
0,@SincerelyLegit Lol why at night? If it's bori...
1,@godtributes Lol
2,@KingBeyonceStan I need to binge on both now b...
3,@_QUEENSharnay Naw I’d i can’t put this skip o...
4,@sannicolaso I have things to do at home but y...


In [12]:
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(tweet_text.text)
word_occurence = tfidf.transform(tweet_text.text).todense()

In [15]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

,gkids,curry,october,purrr,pooped,ijustine,прилетела,بود,eddieg__,acrylics,griffith,superjanella,johnathon,asking,sounded,comic,problem,fishkill,accepts,dylanxgeick
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.decomposition import LatentDirichletAllocation

In [17]:
lda = LatentDirichletAllocation(n_topics=10, learning_method='batch')
lda.fit(word_occurence)

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [18]:
lda_df = pd.DataFrame(lda.components_, columns=words).T

In [19]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [20]:
filter_topic(lda_df, 0, 10)

got     13.484751
just    10.199426
Name: 0, dtype: float64

In [21]:
filter_topic(lda_df, 1, 10)

love     25.632421
like     19.560264
wanna    12.993513
thank    11.043631
Name: 1, dtype: float64

In [22]:
filter_topic(lda_df, 2, 10)

god    10.997544
Name: 2, dtype: float64

In [23]:
filter_topic(lda_df, 3, 10)

good    23.568279
just    22.689027
like    18.980344
lol     13.840254
need    12.848446
know    12.386194
don     12.296953
feel    11.848260
Name: 3, dtype: float64

In [24]:
filter_topic(lda_df, 4, 10)

mood      19.751412
like      17.458468
shit      16.806997
don       16.527893
tell      13.851655
love      13.113227
lmao      12.986063
bitch     12.952206
ain       12.280120
just      11.829047
people    10.855783
really    10.655499
la        10.448887
lol       10.100307
Name: 4, dtype: float64

In [25]:
filter_topic(lda_df, 5, 10)

just    12.737602
day     12.228986
Name: 5, dtype: float64

In [26]:
filter_topic(lda_df, 6, 6)

just         12.518065
time         11.344469
today         9.958814
let           9.664789
sick          8.415636
goodnight     7.701445
did           7.650852
fuck          7.174888
oh            6.404382
gonna         6.372976
like          6.328246
ve            6.040279
Name: 6, dtype: float64

In [27]:
filter_topic(lda_df, 7, 7)

realdonaldtrump    26.950140
fuck               15.390053
just               12.659350
ok                 10.168678
miss               10.007836
thank               9.512851
lol                 9.458856
don                 9.399344
know                9.272409
got                 8.942858
shit                8.454909
like                8.193587
Name: 7, dtype: float64

In [28]:
filter_topic(lda_df, 8, 7)

love      16.295547
sleep     14.787969
just      13.932566
like      12.765349
money     12.255125
really    12.041623
don       11.571354
wow       11.366378
game      10.555745
im        10.228838
know       8.300734
good       8.242944
going      8.232269
want       8.121570
buy        8.111260
lol        8.081380
guys       7.835152
talk       7.534680
day        7.483266
having     7.078854
Name: 8, dtype: float64

In [29]:
filter_topic(lda_df, 9, 7)

gt                 16.415036
hi                 11.229680
lo                 10.158167
ca                 10.111204
love                9.950738
forecast            9.538436
tonight             9.437014
thanks              9.314583
need                8.596261
amp                 8.326719
like                8.220146
right               8.214559
omg                 8.197175
just                8.002618
realdonaldtrump     7.831536
come                7.329148
jan                 7.025839
Name: 9, dtype: float64